#### This code from Medium.com website for demonstration purpose.abs
https://medium.com/getpieces/how-to-build-a-langchain-pdf-chatbot-b407fcd750b9

In [20]:
import os
from PyPDF2 import PdfReader

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.vectorstores import Chroma

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [21]:
# os.environ["OPENAI_API_KEY"] = "your_api_key"


In [22]:
# read in your pdf file

pdf_reader = PdfReader("../uploads/tcs.pdf")

In [4]:
my_text = []
for page in pdf_reader.pages:
     text = page.extract_text()
     my_text.append(text)

In [5]:
text = ' '.join(my_text)
text

'Integrated /A.caltnnual Report 20 years of value creation \n(2004 - 2024)\n2023-2024\n2 \n  TRANSFORMING INDUSTRIES     Integrated Annual Report | 2023-2024\n20 years of creating value for you\nSince our IPO in 2004\n2 \n  TRANSFORMING INDUSTRIES     \nBackdrop: TCS Gitanjali Park Campus, Kolkata, India Tata Consultancy Services is an IT services, consulting and \nbusiness solutions organization that has been partnering with many of the world’s largest businesses in their transformation journeys for over 56 years. Its consulting-led, cognitive powered, portfolio of business, technology and engineering services and solutions is delivered through its unique Location Independent Agile™ delivery model, recognized as a benchmark of excellence in software development.\nA part of the Tata group, India’s largest multinational business \ngroup, TCS has over 601,000 of the world’s best-trained consultants in 54 countries. The company generated consolidated revenues of US$29.1 billion in the fis

In [6]:
len(text) ## token size

1085573

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap  = 32, length_function = len,)
texts = text_splitter.split_text(text)


In [8]:
texts

['Integrated /A.caltnnual Report 20 years of value creation \n(2004 - 2024)\n2023-2024\n2 \n  TRANSFORMING INDUSTRIES     Integrated Annual Report | 2023-2024\n20 years of creating value for you\nSince our IPO in 2004\n2 \n  TRANSFORMING INDUSTRIES     \nBackdrop: TCS Gitanjali Park Campus, Kolkata, India Tata Consultancy Services is an IT services, consulting and',
 'business solutions organization that has been partnering with many of the world’s largest businesses in their transformation journeys for over 56 years. Its consulting-led, cognitive powered, portfolio of business, technology and engineering services and solutions is delivered through its unique Location Independent Agile™ delivery model, recognized as a benchmark of excellence in software development.\nA part of the Tata group, India’s largest multinational business',
 'group, TCS has over 601,000 of the world’s best-trained consultants in 54 countries. The company generated consolidated revenues of US$29.1 billion in th

In [9]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33396\939171391.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [11]:
docsearch = FAISS.from_texts(texts, embeddings)

In [12]:
## We proceed to create a vector database (our knowledge base) using the FAISS library and the OpenAI embeddings.
docsearch

In [13]:
##Once the database has been created, we can then query it.
chain = load_qa_chain(OpenAI(), chain_type="stuff")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33396\3193011437.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  chain = load_qa_chain(OpenAI(), chain_type="stuff")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33396\3193011437.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs

In [23]:
chain

In [15]:

query = "Dividend yield of current year?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33396\4041874025.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


' The dividend yield of the current year cannot be determined based on the given context. We would need more information such as the current stock price of the company and the total number of outstanding shares to calculate the dividend yield.'

In [97]:
## Finally, we get to ask our PDF questions

query = "Which company's annual report it is?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16080\3057820169.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


' TCS (Tata Consultancy Services)'

In [98]:
query = "Who is the CEO of the company?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' N Chandrasekaran'

In [99]:
query = "How much salary CEO takes at last year?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."

In [100]:
query = "What about balancesheet?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The balance sheet shows the financial position of the company at a specific point in time, typically at the end of a fiscal year. It includes the company's assets, liabilities, and shareholder's equity. "

In [101]:
query = "Give me insights from balancesheet."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nFrom the given balance sheet, we can see that the total current liabilities for the company are 43,061 and total equity and liabilities are 1,21,148. The company has a significant amount of investments in the form of bank deposits and earmarked balances with banks, totaling to 2,296 and 29,274 respectively. The company also has a significant amount of trade receivables, both billed and unbilled, which indicates a healthy sales pipeline. The company also has loans and other financial assets, totaling to 1,498 and 1,942 respectively.\n\nOn the other hand, the company has financial liabilities such as trade payables, cash flow hedges, and branch profit tax. The company also has unrealized gains on securities and lease liabilities and right-of-use assets. Overall, the balance sheet shows a healthy mix of assets and liabilities for the company.'

In [103]:
print('\nFrom the given balance sheet, we can see that the total current liabilities for the company are 43,061 and total equity and liabilities are 1,21,148. \nThe company has a significant amount of investments in the form of bank deposits and earmarked balances with banks, totaling to 2,296 and 29,274 respectively. \nThe company also has a significant amount of trade receivables, both billed and unbilled, which indicates a healthy sales pipeline. \nThe company also has loans and other financial assets, totaling to 1,498 and 1,942 respectively.\n\nOn the other hand, the company has financial liabilities such as trade payables, cash flow hedges, and branch profit tax. The company also has unrealized gains on securities and lease liabilities and right-of-use assets. Overall, the balance sheet shows a healthy mix of assets and liabilities for the company.')


From the given balance sheet, we can see that the total current liabilities for the company are 43,061 and total equity and liabilities are 1,21,148. 
The company has a significant amount of investments in the form of bank deposits and earmarked balances with banks, totaling to 2,296 and 29,274 respectively. 
The company also has a significant amount of trade receivables, both billed and unbilled, which indicates a healthy sales pipeline. 
The company also has loans and other financial assets, totaling to 1,498 and 1,942 respectively.

On the other hand, the company has financial liabilities such as trade payables, cash flow hedges, and branch profit tax. The company also has unrealized gains on securities and lease liabilities and right-of-use assets. Overall, the balance sheet shows a healthy mix of assets and liabilities for the company.


In [104]:
query = "From where company earn most?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The company earns the most from North America, with 51.1% of the revenue coming from this region.'

In [16]:
query = "Give me dividend value of current year."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The proposed final dividend for the year ended March 31, 2024 is `28.00 per share, which would result in a cash outflow of approximately `10,131 crore if approved by shareholders at the Annual General Meeting. Additionally, the company has also declared three interim dividends of `9 per equity share and a special dividend of `18 per equity share, totaling `45 per equity share for the current year, involving a cash outflow of `16,355 crore during the year. This brings the total dividend for FY 2024 to `73 per equity share and a total cash outflow of `26,486 crore.'

In [19]:
query = "Give me Dividend of June month"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The dividend declared by the Company for June would depend on the profit available for distribution and the decision of the Board of Directors. The proposed final dividend for the year ended March 31, 2024 is `28.00 per share, subject to approval at the AGM. If approved, payment of the dividend would be made on June 4, 2024. However, the exact amount for June cannot be determined without further information.'